In [ ]:
import tensorflow as tf
import numpy as np
import random
import math

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Reshape, MaxPooling2D
from tensorflow.python.keras.layers import Conv1D, Dense, Flatten, Dropout

## Learning Random Boolean Functions

We define a dynamically produced random Boolean function with the following class, which takes as input to its call function a {0, 1} float valued vector:

In [ ]:
class RandomFunction:
    def __init__(self):
        self.fn = dict()

    def __call__(self, x):
        k = str(list(x))
        if k not in self.fn:
            self.fn[k] = random.choice([float(0), float(1)])
        return self.fn[k]
    
f = RandomFunction()

Next, we set a number of bits for the function to take in, as well as the training and test sets.

In [ ]:
def bitstrings(n):
    if (n == 0):
        return [[]]
    else:
        bs = bitstrings(n - 1)
        return list(map(lambda b: [0] + b, bs)) + list(map(lambda b: [1] + b, bs))

n_bits = 40

n_training_points = 100*n_bits 

# training_data = [[random.choice([float(0), float(1)]) for _ in range(n_bits)] for _ in range(n_training_points)]

training_data = bitstrings(n_bits)

training_input = np.array(training_data)

training_output = np.apply_along_axis(f, 1, training_input)

n_test_points = 100*n_bits

# test_data = [[random.choice([float(0), float(1)]) for _ in range(n_bits)] for _ in range(n_test_points)]

test_data = bitstrings(n_bits)

test_input = np.array(test_data)

test_output = np.apply_along_axis(f, 1, test_input)

Finally, we build the model! It is a feed-forward, or sequential, network with two RELU layers and a softmax layer.

In [ ]:
model = Sequential()
model.add(Dense(n_bits, activation='relu'))
model.add(Dropout(1/2))
model.add(Dense(n_bits, activation='tanh'))
model.add(Dropout(1/2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='SGD', metrics=['accuracy'])

We evaluate its initial performance.

In [ ]:
err0 = model.evaluate(test_input, test_output, verbose=0)[1]

Then, at last, we train the model and report how much better it got.

In [ ]:
model.fit(training_input, training_output, epochs=100, verbose=0)

errf = model.evaluate(test_input, test_output, verbose=0)[1]
print("Final test error: {}".format(errf))
print("Original test error: {}".format(err0))
print("Difference: {}".format(errf - err0))